# Regularization

## Load data

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

pickle_file = './data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


## Reformate data
- data as a flat matrix [batch size x 28 x 28] $\rightarrow$ [batch size x 784],
- labels as float 1-hot encodings.

In [2]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


## Define the accuracy function

In [3]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## L2 regularization
L2 amounts to adding a penalty on the norm of the weights to the loss.

L2 for logistic model

In [4]:
batch_size = 128
reg_weight = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Add L2 regulization
  loss += reg_weight*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Initialized
Minibatch loss at step 0: 23.542894
Minibatch accuracy: 7.0%
Validation accuracy: 6.7%
Minibatch loss at step 500: 2.442786
Minibatch accuracy: 80.5%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 1.959828
Minibatch accuracy: 75.8%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 1.029323
Minibatch accuracy: 83.6%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 0.863023
Minibatch accuracy: 84.4%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 0.865065
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 0.811852
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Test accuracy: 88.8%


## L2 for 1 hidden layer neural network

In [5]:
batch_size = 128
num_hidden_nodes = 1024 
reg_weight = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Hidden layer
  weights_h = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_h = tf.Variable(tf.zeros([num_hidden_nodes]))

  tf_train_h = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h)+biases_h)
  tf_valid_h = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h)+biases_h)
  tf_test_h = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h)+biases_h)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_h, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Add L2 regulization
  loss += reg_weight*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights_h))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_h, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_h, weights) + biases)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 648.573242
Minibatch accuracy: 3.9%
Validation accuracy: 28.9%
Minibatch loss at step 500: 198.838165
Minibatch accuracy: 79.7%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 114.119133
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 68.195244
Minibatch accuracy: 89.8%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 41.103668
Minibatch accuracy: 90.6%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.160469
Minibatch accuracy: 88.3%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.440619
Minibatch accuracy: 86.7%
Validation accuracy: 86.5%
Test accuracy: 93.2%


## Restrict training data to a few batches
Result in overfitting

In [6]:
batch_size = 128
num_hidden_nodes = 1024 

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Hidden layer
  weights_h = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_h = tf.Variable(tf.zeros([num_hidden_nodes]))

  tf_train_h = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h)+biases_h)
  tf_valid_h = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h)+biases_h)
  tf_test_h = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h)+biases_h)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_h, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_h, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_h, weights) + biases)

num_steps = 3001
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % num_batches*batch_size
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 303.959534
Minibatch accuracy: 8.6%
Validation accuracy: 20.2%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Test accuracy: 80.4%


## Introduce Dropout on the hidden layer
Dropout should only be introduced during training, not evaluation.

In [7]:
batch_size = 128
num_hidden_nodes = 1024 
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Hidden layer
  weights_h = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_h = tf.Variable(tf.zeros([num_hidden_nodes]))

  tf_train_h = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h)+biases_h)
  tf_valid_h = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h)+biases_h)
  tf_test_h = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h)+biases_h)
    
  # Dropout for training
  tf_train_h_drop = tf.nn.dropout(tf_train_h, keep_prob)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_h_drop, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_h, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_h, weights) + biases)

num_steps = 3001
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % num_batches*batch_size
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 499.112610
Minibatch accuracy: 9.4%
Validation accuracy: 23.2%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Test accuracy: 85.6%


## Improve performance
- Multi-layer 
- L2 regularization
- Learning rate decay

In [9]:
batch_size = 128
num_h1_nodes = 1024 
num_h2_nodes = 1024
num_h3_nodes = 1024
num_h4_nodes = 1024

reg_weight = 0.003
reg = 0
keep_prob_value = 1

decay_steps = 300
decay_rate = 0.9
starter_learning_rate = 0.5

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def build_relu_layer(tf_dataset, weights, biases):
  return  tf.nn.relu(tf.matmul(tf_dataset, weights)+biases)

graph = tf.Graph()
with graph.as_default():
   
  # specify input
  tf_dataset = tf.placeholder(tf.float32, shape = (None, image_size * image_size))
  tf_labels = tf.placeholder(tf.float32, shape = (None, num_labels))
  keep_prob = tf.placeholder(tf.float32)
    
  # Hidden layer 1
  weights_h1 = weight_variable([image_size * image_size, num_h1_nodes])
  biases_h1 = bias_variable([num_h1_nodes])
  
  tf_dataset_h1 = build_relu_layer(tf_dataset, weights_h1, biases_h1)
  reg += reg_weight * tf.nn.l2_loss(weights_h1) 
    
  # Add dropout
  tf_dataset_h1_drop = tf.nn.dropout(tf_dataset_h1, keep_prob)
  
  # Hidden layer 2
  weights_h2 = weight_variable([num_h1_nodes, num_h2_nodes])
  biases_h2 = bias_variable([num_h2_nodes])
  
  tf_dataset_h2 = build_relu_layer(tf_dataset_h1_drop, weights_h2, biases_h2)
  reg += reg_weight * tf.nn.l2_loss(weights_h2)

  # Add dropout
  tf_dataset_h2_drop = tf.nn.dropout(tf_dataset_h2, keep_prob)

  # Hidden layer 3
  weights_h3 = weight_variable([num_h2_nodes, num_h3_nodes])
  biases_h3 = bias_variable([num_h3_nodes])
    
  tf_dataset_h3 = build_relu_layer(tf_dataset_h2_drop, weights_h3, biases_h3)
  reg += reg_weight * tf.nn.l2_loss(weights_h3)

  # Add dropout
  tf_dataset_h3_drop = tf.nn.dropout(tf_dataset_h3, keep_prob)
    
  # Hidden layer 4
  weights_h4 = weight_variable([num_h3_nodes, num_h4_nodes])
  biases_h4 = bias_variable([num_h4_nodes])
    
  tf_dataset_h4 = build_relu_layer(tf_dataset_h3_drop, weights_h4, biases_h4)
  reg += reg_weight * tf.nn.l2_loss(weights_h4)

  # Add dropout
  tf_dataset_h4_drop = tf.nn.dropout(tf_dataset_h4, keep_prob)
    
  # output layer
  weights_h = weight_variable([num_h4_nodes, num_labels])
  biases_h = bias_variable([num_labels])

  logits = tf.matmul(tf_dataset_h4_drop, weights_h) + biases_h
  reg += reg_weight * tf.nn.l2_loss(weights_h)
  
  
  # Training computation.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=logits))
    
  # Add L2 regulization
  loss += reg
  
  # Optimizer, learning rate decay
  # decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 
                                             decay_steps, decay_rate, staircase = True)
    
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(
      loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits)

num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels, keep_prob : keep_prob_value}
    _, l, train_predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(train_predictions, batch_labels))
      valid_predictions = prediction.eval(
          {tf_dataset : valid_dataset, tf_labels : valid_labels, keep_prob : 1.0})
      print("Validation accuracy: %.1f%%" % accuracy(valid_predictions, valid_labels))
  test_predictions = prediction.eval({tf_dataset : test_dataset, tf_labels : test_labels, keep_prob : 1.0})
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))

Initialized
Minibatch loss at step 0: 2.764119
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.660231
Minibatch accuracy: 86.7%
Validation accuracy: 83.0%
Minibatch loss at step 1000: 0.713150
Minibatch accuracy: 83.6%
Validation accuracy: 84.6%
Minibatch loss at step 1500: 0.499543
Minibatch accuracy: 89.8%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 0.465141
Minibatch accuracy: 92.2%
Validation accuracy: 86.4%
Minibatch loss at step 2500: 0.544397
Minibatch accuracy: 89.1%
Validation accuracy: 86.8%
Minibatch loss at step 3000: 0.625394
Minibatch accuracy: 84.4%
Validation accuracy: 86.7%
Minibatch loss at step 3500: 0.602515
Minibatch accuracy: 86.7%
Validation accuracy: 87.5%
Minibatch loss at step 4000: 0.507384
Minibatch accuracy: 89.8%
Validation accuracy: 87.4%
Minibatch loss at step 4500: 0.505974
Minibatch accuracy: 90.6%
Validation accuracy: 87.4%
Minibatch loss at step 5000: 0.549862
Minibatch accuracy: 89.8%
Validation accurac